In [1]:
from NextVisionML import MLContext
from NextVisionML import VarianceFilter, DecisionTreeClfr, OneHotClfr, CorrelationFilter, MutualInfoFilter

In [2]:
#Create an context (Loads data)
mlcontext = MLContext()

In [3]:
#add feature selection methods
mlcontext.train_preparation_methods.append(VarianceFilter(mlcontext)) 
mlcontext.train_preparation_methods.append(CorrelationFilter(mlcontext)) 
mlcontext.train_preparation_methods.append(MutualInfoFilter(mlcontext)) 
#Add data prepocessing methods
mlcontext.train_methods.append(OneHotClfr(mlcontext))
mlcontext.train_methods.append(DecisionTreeClfr(mlcontext))
#select a model type
#start the training process
mlcontext.start_train_process()

  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

c:\Program Files\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 50%|█████     | 1/2 [00:29<00:29, 29.96s/trial, best loss: 0.7887276785714286]

c:\Program Files\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

c:\Program Files\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 50%|█████     | 1/2 [00:28<00:28, 28.46s/trial, best loss: 0.8009300595238096]

c:\Program Files\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



100%|██████████| 2/2 [01:07<00:00, 33.57s/trial, best loss: 0.8009300595238096]


In [4]:
model = mlcontext.train_methods[0].model


In [5]:
from sklearn.model_selection import cross_val_score

X = mlcontext.test_X
y = mlcontext.test_y

test = model.predict(X)

In [6]:
import pickle

pickle.dumps(mlcontext.train_methods[0].model.net)

b'\x80\x04\x95\xc5\x00\x00\x00\x00\x00\x00\x00\x8c.NextVisionML.train.interfaces.train.OneHotClfr\x94\x8c\x10SimpleNetSoftmax\x94\x93\x94)\x81\x94\x8c\x0bcollections\x94\x8c\x0bOrderedDict\x94\x93\x94)R\x94(\x8c\x0bfc.0.weight\x94\x8c\x0ctorch._utils\x94\x8c\x12_rebuild_tensor_v2\x94\x93\x94(\x8c\rtorch.storage\x94\x8c\x10_load_from_bytes\x94\x93\x94B\xc2\xac\x01\x00\x80\x02\x8a\nl\xfc\x9cF\xf9 j\xa8P\x19.\x80\x02M\xe9\x03.\x80\x02}q\x00(X\x10\x00\x00\x00protocol_versionq\x01M\xe9\x03X\r\x00\x00\x00little_endianq\x02\x88X\n\x00\x00\x00type_sizesq\x03}q\x04(X\x05\x00\x00\x00shortq\x05K\x02X\x03\x00\x00\x00intq\x06K\x04X\x04\x00\x00\x00longq\x07K\x04uu.\x80\x02(X\x07\x00\x00\x00storageq\x00ctorch\nFloatStorage\nq\x01X\r\x00\x00\x001471988793488q\x02X\x03\x00\x00\x00cpuq\x03M\xf2jNtq\x04Q.\x80\x02]q\x00X\r\x00\x00\x001471988793488q\x01a.\xf2j\x00\x00\x00\x00\x00\x00\xdd>\xb0=\xe1\x8d\x9f<\'\x85y\xbd\x8d\x14\x9f\xbcUG.\xbc\x89<\xa7<7\x02Q\xbd\xee\x13\xaf\xbd\x8c\xf8O=\x97\xda\xca<\'\x85\xb

In [7]:
type(mlcontext.train_methods[0].model)

NextVisionML.train.interfaces.train.OneHotClfr.OneHotClfrCustom